In [6]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Attribute information

Each record is an example of a hand consisting of five playing cards drawn from a standard deck of 52. Each card is described using two attributes (suit and rank), for a total of 10 predictive attributes.

* ranks (C1, C2, C3, C4, C5)    
  - 1: Ace  
  - 2-10: 2-10  
  - 11: Jack  
  - 12: Queen  
  - 13: King  
* suits (S1, S2, S3, S4, S5)   
  - 1: Hearts  
  - 2: Spades  
  - 3: Diamonds  
  - 4: Clubs  
* hand  
  - 0: Nothing in hand; not a recognized poker hand 
  - 1: One pair; one pair of equal ranks within five cards
  - 2: Two pairs; two pairs of equal ranks within five cards
  - 3: Three of a kind; three equal ranks within five cards
  - 4: Straight; five cards, sequentially ranked with no gaps
  - 5: Flush; five cards with the same suit
  - 6: Full house; pair + different rank three of a kind
  - 7: Four of a kind; four equal ranks within five cards
  - 8: Straight flush; straight + flush
  - 9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush

# Get the dataset

In [8]:
# dataset location
testing_data_location = "./dataset/poker-hand-testing.data"
training_data_location = "./dataset/poker-hand-training-true.data"

# assign column names to the dataset attributes
names = ['S1','C1','S2','C2','S3','C3','S4','C4','S5','C5','hand']

# get testing & training data
testing = pd.read_csv(testing_data_location, names=names)
training = pd.read_csv(training_data_location, names=names)

# show the first five columns of the training hand
print("Training Hand")
training.head()

Training Hand


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,1,10,1,11,1,13,1,12,1,1,9
1,2,11,2,13,2,10,2,12,2,1,9
2,3,12,3,11,3,13,3,10,3,1,9
3,4,10,4,11,4,1,4,13,4,12,9
4,4,1,4,13,4,12,4,11,4,10,9


In [9]:
# show the first five columns of the testing hand
print("Testing Hand")
testing.head()

Testing Hand


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0


# Preprocessing

### separate dataset attributes (X) from Class (Y)

In [39]:
# separate first ten columns (X) from eleven'th column for training set
X_train = training.drop(['hand'],axis=1)
Y_train = training.hand

# ...for testing set
X_test = testing.drop(['hand'],axis=1)
Y_test = testing.hand

# show first five columns of training X
print("training X")
X_train.head()

training X


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,1,10,1,11,1,13,1,12,1,1
1,2,11,2,13,2,10,2,12,2,1
2,3,12,3,11,3,13,3,10,3,1
3,4,10,4,11,4,1,4,13,4,12
4,4,1,4,13,4,12,4,11,4,10


In [35]:
# show first five columns of training Y
print("training Y")
Y_train.head()

training Y


0    9
1    9
2    9
3    9
4    9
Name: hand, dtype: int64

### show unique Class values

In [42]:
# show unique Class categories
Y_train.unique()

array([9, 8, 1, 0, 4, 3, 2, 5, 6, 7])

# Training and Predictions

In [49]:
# create 3 layers of 10 nodes each + 1000 iterations
mlp = MLPClassifier(solver='adam', hidden_layer_sizes=(64, 64), max_iter=2000, activation='tanh', learning_rate_init=0.02)

# train algorithm on training data
mlp.fit(X_train, Y_train.values.ravel())

# make predictions
predictions = mlp.predict(X_test)

# Check the results

In [50]:
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test, predictions))

[[497369   1770      0      0   1650    367      0      0     49      4]
 [  3681 417822    472     87    267     42      0      0    126      1]
 [     0   1323  45900    388      0      0      9      2      0      0]
 [     0      0    599  20488      0      0     10     24      0      0]
 [  2771     86      0      0   1003      0      0      0     24      1]
 [  1629      4      0      0      9    338      0      0     13      3]
 [     0      0      7    750      0      0    662      5      0      0]
 [     0      0      0     90      0      0     87     53      0      0]
 [     8      0      0      0      3      1      0      0      0      0]
 [     1      0      0      0      0      2      0      0      0      0]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99    501209
           1       0.99      0.99      0.99    422498
           2       0.98      0.96      0.97     47622
           3       0.94      0.97      0.95     2112